# Desafio Final
---
O projeto consiste em uma conciliação do valor total de vendas de recarga de celular por operadora e por loja. A Bemol recebe um arquivo de uma empresa financeira (detalhamento_m4u.xlsx) onde possui o valor consolidado de vendas e o objetivo é verificar se os dados do SAP (mcg3.csv) estão de acordo com o informado. Gerar um alerta caso os valores estejam divergentes.

Dica:
Os dados do SAP vêm separados por cada transição de venda realizada. A ideia é de agrupar esses dados para deixá-los no formato parecido com que recebemos da empresa externa e, assim, poder unificar as tabelas e fazer a comparação.

## Importação das bibiotecas

In [1]:
import pandas as pd


## Importação dos Arquivos

### Importação dos arquivos das lojas

In [2]:
df_am = pd.read_excel('detalhamento_m4u.xlsx',header=1,sheet_name = 'AM')
df_rr = pd.read_excel('detalhamento_m4u.xlsx',header=1,sheet_name = 'RR')
df_ro = pd.read_excel('detalhamento_m4u.xlsx',header=1,sheet_name = 'RO')
df_ac = pd.read_excel('detalhamento_m4u.xlsx',header=1,sheet_name = 'AC')

### Importação do arquivo base interno.

In [3]:
df_interno = pd.read_csv('mcg3.csv',sep='|')

## Tratamento Dados

### Dados das Lojas

#### Concateação dos dataframes

In [4]:
df_lojas = pd.concat([df_am,df_rr,df_rr,df_ro])

#### Renomeação das Colunas

In [5]:
df_lojas.columns = ['nome','loja','uf','operadora','valor_bruto']

#### Colocar valore coluna operadora em uppercase

In [6]:
df_lojas['operadora'] = df_lojas['operadora'].str.upper()

#### Verificando e convertendo tipos se necessário

In [7]:
df_lojas.dtypes

nome           object
loja            int64
uf             object
operadora      object
valor_bruto     int64
dtype: object

In [8]:
df_lojas['loja'] = df_lojas['loja'].astype('str')

In [9]:
 df_lojas['valor_bruto'] = df_lojas['valor_bruto'].astype('float')

#### Eliminando dados

In [10]:
df_lojas = df_lojas.drop(['nome','uf'],axis='columns')

In [11]:
df_lojas.head(5)

,loja,operadora,valor_bruto
0,103,CLARO,2651.0
1,103,OI,3309.0
2,103,TIM,1550.0
3,103,VIVO,4739.0
4,107,CLARO,1495.0


#### Agrupando dados

In [12]:
df_lojas.info() #verificando dados anstes do agrupamento

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 23
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loja         166 non-null    object 
 1   operadora    166 non-null    object 
 2   valor_bruto  166 non-null    float64
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


In [13]:
sum(df_lojas['valor_bruto']) #verificando a soma total antes do agrupamento

168359.0

In [14]:
df_lojas = df_lojas.groupby(['loja','operadora']).agg({'valor_bruto':'sum',}).reset_index()

In [15]:
df_lojas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loja         158 non-null    object 
 1   operadora    158 non-null    object 
 2   valor_bruto  158 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.8+ KB


In [16]:
sum(df_lojas['valor_bruto'])

168359.0

### Dados base interno

#### Renomeação das colunas

In [17]:
df_interno.columns = ['loja','operadora','material','data','valor_liquido']

#### Colocar valore coluna operadora em uppercase

In [18]:
df_interno['operadora'] = df_interno['operadora'].str.upper()

#### Verificando tipos e convertendo se necessário

In [19]:
df_interno.dtypes

loja               int64
operadora         object
material          object
data              object
valor_liquido    float64
dtype: object

In [20]:
df_interno['loja']=df_interno['loja'].astype('str')

#### Eliminando dados

In [21]:
df_interno = df_interno.drop(['material','data'],axis='columns')

In [22]:
df_interno.head(5)

,loja,operadora,valor_liquido
0,101,VIVO,20.0
1,101,VIVO,10.0
2,101,VIVO,20.0
3,101,VIVO,10.0
4,101,VIVO,25.0


#### Agrupando dados

In [23]:
df_interno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5113 entries, 0 to 5112
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   loja           5113 non-null   object 
 1   operadora      5113 non-null   object 
 2   valor_liquido  5113 non-null   float64
dtypes: float64(1), object(2)
memory usage: 120.0+ KB


In [24]:
sum(df_interno['valor_liquido'])

166880.13999999998

In [25]:
df_interno = df_interno.groupby(['loja','operadora']).agg({'valor_liquido':'sum'}).reset_index()

In [26]:
df_interno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   loja           163 non-null    object 
 1   operadora      163 non-null    object 
 2   valor_liquido  163 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.9+ KB


In [27]:
sum(df_interno['valor_liquido'])

166880.13999999998

## Preparando o relatório

### Merge dos dataframes

In [28]:
df_relatorio = pd.merge(df_lojas,df_interno,on=['loja','operadora'],how='outer')

In [29]:
df_relatorio.head(5)

,loja,operadora,valor_bruto,valor_liquido
0,101,CLARO,1527.0,1527.0
1,101,OI,1730.0,NaN
2,101,TIM,975.0,975.0
3,101,VIVO,3142.0,3142.0
4,103,CLARO,2651.0,2652.3


### Tratamento pós merge

In [30]:
df_relatorio = df_relatorio.fillna(0)

### Implementação da regra de negocio

#### Definição da regra de negocio

>Se houver diferença entre os valores liquido e bruto mostrar mensagem de alerta

#### Implementação da regra de negocio

In [31]:
def diferenca(liquido,bruto):
    return "alerta" if liquido!=bruto else ""

#### Aplicação da regra de negocio


In [32]:
df_relatorio['status'] = df_relatorio.apply(lambda row: diferenca(row['valor_liquido'],row['valor_bruto']),axis='columns')

In [33]:
df_relatorio

,loja,operadora,valor_bruto,valor_liquido,status
0,101,CLARO,1527.0,1527.0,
1,101,OI,1730.0,0.0,alerta
2,101,TIM,975.0,975.0,
3,101,VIVO,3142.0,3142.0,
4,103,CLARO,2651.0,2652.3,alerta
...,...,...,...,...,...
198,615,TNL,0.0,225.0,alerta
199,616,TNL,0.0,95.0,alerta
200,617,TNL,0.0,525.0,alerta
201,618,TNL,0.0,255.0,alerta


### Filtrando Resultado

In [34]:
df_relatorio_filtro = df_relatorio[(df_relatorio['status'] == 'alerta')]
df_relatorio_filtro

,loja,operadora,valor_bruto,valor_liquido,status
1,101,OI,1730.0,0.0,alerta
4,103,CLARO,2651.0,2652.3,alerta
5,103,OI,3309.0,0.0,alerta
9,105,OI,2030.0,0.0,alerta
13,106,OI,1895.0,0.0,alerta
...,...,...,...,...,...
198,615,TNL,0.0,225.0,alerta
199,616,TNL,0.0,95.0,alerta
200,617,TNL,0.0,525.0,alerta
201,618,TNL,0.0,255.0,alerta


### Exportando Relatório

#### Exportando arquivos csv

In [35]:
df_relatorio.to_csv("Relatório Completo.csv",encoding='utf-8',index=False)
df_relatorio_filtro.to_csv("Relatório somente alertas.csv",encoding='utf-8',index=False)

#### criando arquivos Excel

In [36]:
relatorioExcel = pd.ExcelWriter("Relatório.xlsx",engine='xlsxwriter')

#### Exportando para o Excel

In [37]:
df_relatorio.to_excel(relatorioExcel,sheet_name="Geral",index=False)
df_relatorio_filtro.to_excel(relatorioExcel,sheet_name="Alertas",index=False)

#### Salvando Arquivos

In [38]:
relatorioExcel.save()